# Laundry Symbol Recognition using Convolutional Neural Networks

## Welcome
In this hands-on session we will guide you through the following content:
- Image data visualization
- Image data augmentation
- Training your own CNN based model for image recognition
- Optimizing the performance of your model
- Applying the trained model to classify images 

## Image data visualization
Before you start to train your deep learning models, it is always a good practice to get some intuition about your data at first.

But before that, let's import some libraries that we need.

In [ ]:
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import os
import glob 
import shutil
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.03
set_session(tf.Session(config=config))

%matplotlib inline

The following code cell will display the first image sample from each class in our training set: 

In [ ]:
fig=plt.figure(figsize=(10, 10))
columns = 4
rows = 3
for i in range(1, len(next(os.walk('data/train'))[1])+1): 
    folder_name = next(os.walk('data/train'))[1][i-1]
    img = Image.open(glob.glob('data/train/' + folder_name + '/*.tif')[0], 'r')
    fig.add_subplot(rows, columns, i, title=folder_name)
    plt.imshow(img)
plt.show()

## Image data augmentation

For most practical deep learning applications, overfitting is one of the most common issues which need to be overcome.

>In statistics, overfitting is "the production of an analysis that corresponds too closely or exactly to a particular set of data, and may therefore fail to fit additional data or predict future observations reliably". 

--- Wikipedia

A very common yet effective technique to avoid overfitting is to use data augmentation.

By using ImageDataGenerator class from keras, image data can be augmented on the fly. Let's import it first:  

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

By running the following code cell, we will load and display a single image from our training set. This will be the 'seed image' for the coming data augmentation: 

In [ ]:
img = load_img('data/train/1_waschen30_1_96/Bilderkennung_00003_Waschen_96.tif')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (96, 96, 3)
fig=plt.figure(figsize=(3, 3))
fig.add_subplot(111, title='Original')
plt.imshow(x/255.)
plt.show()

To save images generated by the ImageDataGenerator, we need to create a new folder called "augmented_images":

In [ ]:
try:
    os.mkdir('augmented_images')
except FileExistsError:
    # if directory already exists
    shutil.rmtree('augmented_images')
    os.mkdir('augmented_images')

Define the augmentation needed in the following cell (feel free to modify the parameters and re-run the next 3 cells to see the effects): 

In [ ]:
augmentation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,
    shear_range=0.2, 
    zoom_range=0.2,
    width_shift_range=0.2, 
    height_shift_range=0.2
    )

The ImageDataGenerator.flow() method takes 4-D "batched" image data with form (batch_size, img_height, img_width, #channels) as input, so the original 3-D images array must be "extended" with one additional dimention:

In [ ]:
x_reshaped = x.reshape((1,) + x.shape) # this now a Numpy array with shape (1, img_height, img_width, #channels)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `augmented_images` directory
i = 0
for batch in augmentation_datagen.flow(x_reshaped, batch_size=1,
                          save_to_dir='augmented_images', save_prefix='augmented_', save_format='jpg'):
    i += 1
    if i > 8:
        break  # otherwise the generator would loop indefinitely

Let's visualize the augmented images:

In [ ]:
fig=plt.figure(figsize=(10, 10))
fig.suptitle('Augmented Images')
columns = 3
rows = 3
for i in range(1, columns*rows +1):
    img = Image.open(glob.glob('augmented_images' + '/*.jpg')[i-1], 'r')
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

## Training your own CNN based model for image classification

Now we can finally start training our CNN model! To do so, again we need to import the necessary libraries first:

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, BatchNormalization

Although one great thing about machine learning is that the model parameters can be "automatically" learned during training, we still need to configure some parameters manually. Typically, these parameters include data batch size, number of network layers and number of neurons in each layer... 

We call this kind of parameters "hyperparameters".

In our case, the input image size and batch size belong to them. So, let's define them:

In [ ]:
img_height, img_width = 96, 96
batch_size = 32

To load the training data as well as the validation data for our training, we'll use the following two ImageDataGenerators, respectively. 

To be noticed, we'll convert the images into grayscale images for computing efficiency reason. 

In [ ]:
train_data_dir = 'data/train'
validation_data_dir = 'data/val'

train_datagen = ImageDataGenerator(
        rescale=1./255,
#        shear_range=0.1, 
#        zoom_range=0.1,
#        rotation_range=10,
        )

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    seed=8
    )

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    seed=8
    )

Now we need to build our neural network model. 

With keras, it can be pretty much like building legos: you just stack the parts you want, layer by layer (here I just write the code a little bit explicitly on purpose, to make it more self-explainary).

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=[img_height, img_width, 1]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
          
model.add(Dense(train_generator.num_classes))
model.add(Activation('softmax'))

After building the model, you also need to configure it with the method "compile". Typically, here you want to choose the loss function, optimizer and the metrics for training the model: 

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Another hyperparameter we want to define is the number of training epochs. One epoch is one training iteration over the entire dataset:

In [ ]:
nb_train_samples = train_generator.n # number of training samples
nb_validation_samples = validation_generator.n # number of validation samples
nb_epochs = 10 # number of training epochs

Now we'll begin training our model on data generated batch-by-batch using ImageDataGenerator. After finishing every training epoch, one validation on our validation set will be triggered as well. 

You can observe the metric values during the training: 

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size + 1,
        epochs=nb_epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size + 1,
        )

If you have noticed, the validation loss somehow didn't decrease anymore during the last few epochs, it indicates that the model has been fitted to the data. Otherwise you should enlarge the number of training epochs. 

## Evaluate your model

Since the model has been trained, now it's time for us to test it's final (generalization) performance. To do so, we need to use our test set:

In [ ]:
test_data_dir = 'data/test'

test_generator = val_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        color_mode='grayscale',
        batch_size=batch_size,
        shuffle=False,
        class_mode=None) # no labels for predict_generator!

Also, we need to import some evaluation tools to help us:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

We now start the model evaluation by:
1. letting it make class predictions on all test samples
2. comparing the predictions with ground truth we can get several metrics indicating the model performance

In [ ]:
Y_pred = model.predict_generator(test_generator, test_generator.n // batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1) # indice of class with the highest confidence

Calculate accuracy:

In [ ]:
nb_pred_correct = np.sum(test_generator.classes==y_pred)
print('Test Accuracy: {}%'.format(nb_pred_correct/test_generator.n*100.0))

Display the confusion matrix:

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

Display the classification report:

In [ ]:
print('Classification Report')
target_names = list(test_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

### Voluntary task: Are you satisfied with this result? If not, how can you improve it?

In [ ]:
def predict_on_single_img(model, img, label_dict):
    x = img_to_array(img)/255. # rescale
    x = np.expand_dims(x, axis=0) # extend dimesion
    y_pred = model.predict(x) 
    class_pred = y_pred[0].argmax(axis=-1) 
    return label_dict[class_pred], max(y_pred[0])

If you are now satisfied with your model performance, you can proceed with the remaining part:

To verify and get some feeling about the model you have just trained, we would like to output some the predictions of our test samples as well as compare them of the ground truth:

To output the predictions in a human-readable form, we need a dictionary to map the model output to label text: 

In [ ]:
label_dict = {y:x for x,y in test_generator.class_indices.items()} # {class number : text label}

Now you can run the code cell below to see the result and convince yourself of the discriminative power of CNN:)

In [ ]:
fig=plt.figure(figsize=(15, 20))
columns = 2
rows = 5
for i in range(1, len(next(os.walk('data/test'))[1])+1): 
    folder_name = next(os.walk('data/test'))[1][i-1]
    img = glob.glob('data/test/' + folder_name + '/*.tif')[0]
    img = load_img(img, target_size=(img_height, img_width), color_mode='grayscale')
    pred, confidence = predict_on_single_img(model, img, label_dict)
    fig.add_subplot(rows, columns, i, 
                    title='True: {} / Pred.: {} / Conf.: {:.4f}'.format(folder_name, pred, confidence))
    plt.imshow(img, cmap='gray')      
plt.show()

## Summary
In this hands-on training, you have learned:

- How to build a image dataset
- How to augment image data to avoid overfitting
- How to train a convolutional neural network from scratch
- How to evaluate the trained model
- ### And, most importantly, experience the power and fun of deep learning by yourself:) 

# If you are eager to learn even more about the topic...
* [Machine Learning is Fun! Part 3: Deep Learning and Convolutional Neural Networks](https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721) - A more in-detailed introduction to convolutional neural networks.
* [Deep Learning with Python](https://www.amazon.de/Deep-Learning-with-Python/dp/B07H5TKXHN/ref=sr_1_6?__mk_de_DE=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=26ZPNVY500LY2&keywords=deep+learning+with+python&qid=1560441999&s=books-intl-de&sprefix=deep+learning+with+p%2Caps%2C291&sr=1-6) - Very good book with intuitive explanations and practical examples! By the creator of keras.